In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
directory = '../4 HAND GESTURE/images'

In [5]:
Name=[]
for file in os.listdir(directory):
    if file[-4:]!='pt.m' and file[-4:]!='.txt':
        Name+=[file]
print(Name)
print(len(Name))

['call_me', 'fingers_crossed', 'okay', 'paper', 'peace', 'rock', 'rock_on', 'scissor', 'thumbs', 'up']
10


In [7]:
File=[]
for file in os.listdir(directory):
    File+=[file]
    print(file)

call_me
fingers_crossed
okay
paper
peace
rock
rock_on
scissor
thumbs
up


In [9]:
dataset=[]
testset=[]
count=0
for file in File:
    path=os.path.join(directory,file)
    t=0
    for im in os.listdir(path):
        if im[-4:]!='pt.m' and im[-4:]!='.txt':
            image=load_img(os.path.join(path,im), color_mode='rgb', target_size=(60,60))
            image=img_to_array(image)
            image=image/255.0
            if t<400:
                dataset.append([image,count])
            else:   
                testset.append([image,count])
            t+=1
    count=count+1

In [10]:
data,labels0=zip(*dataset)
test,tlabels0=zip(*testset)

In [11]:
labels1=to_categorical(labels0)
data=np.array(data)
labels=np.array(labels1)

In [12]:
tlabels1=to_categorical(tlabels0)
test=np.array(test)
tlabels=np.array(tlabels1)

In [13]:
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

In [14]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [15]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(60,60,3),include_top=False,weights='imagenet',pooling='avg')

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 38s 1us/step


In [16]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(10, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [17]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [18]:
HG = model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=20)

Epoch 1/20


c:\Users\Goks\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 200s 723ms/step - accuracy: 0.3894 - loss: 1.7799 - val_accuracy: 0.2675 - val_loss: 11.7660
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 61s 607ms/step - accuracy: 0.7481 - loss: 0.7896 - val_accuracy: 0.7362 - val_loss: 2.2817
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 61s 607ms/step - accuracy: 0.8225 - loss: 0.5702 - val_accuracy: 0.9275 - val_loss: 0.3103
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 62s 616ms/step - accuracy: 0.8821 - loss: 0.3852 - val_accuracy: 0.9413 - val_loss: 0.2066
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 62s 621ms/step - accuracy: 0.8873 - loss: 0.3627 - val_accuracy: 0.8600 - val_loss: 0.6788
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 61s 607ms/step - accuracy: 0.9011 - loss: 0.3400 - val_accuracy: 0.9175 - val_loss: 0.2412
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 61s 608ms/step - accuracy: 0.9222 - loss: 0.2777 - val_accuracy: 0.9475 - val_loss: 0.1643
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 62s 616ms/step - accuracy: 0.9373 - loss: 0.2141 - v

In [19]:
y_pred=model.predict(testx)

25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 210ms/step


In [23]:
image=load_img("../4 HAND GESTURE/images/rock_on/1317.jpg",target_size=(60,60))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [25]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [26]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print("Prediction is {}.".format(move_name))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Prediction is rock_on.


In [27]:
model.save('handgest.hdf5')
model2 = keras.models.load_model('handgest.hdf5')